In [34]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as TF
import pandas as pd
import torch


x_train_data = pd.read_csv(".\\samples\\train\\jupyter_landmarks.csv",
                           usecols = range(1, 209))

y_train_data = pd.read_csv(".\\samples\\train\\unity_blendshapes.csv",
                           usecols = range(2, 74))


x_train_tensor = torch.tensor(x_train_data.values, requires_grad=True).float()

for i in range(x_train_tensor.shape[1]):
    sdev_norm = torch.std(x_train_tensor[:, i])
    sdev_norm = 0.001 if sdev_norm == 0 else sdev_norm
    x_train_tensor[:, i] = (x_train_tensor[:, i] - torch.mean(x_train_tensor[:, i])) / sdev_norm

y_train_tensor = torch.tensor(y_train_data.values, requires_grad=True).float()


for i in range(y_train_tensor.shape[1]):
    sdev_norm = torch.std(y_train_tensor[:, i])
    sdev_norm = 0.001 if sdev_norm == 0 else sdev_norm
    y_train_tensor[:, i] = (y_train_tensor[:, i] - torch.mean(y_train_tensor[:, i])) / sdev_norm



x_test_data = pd.read_csv(".\\samples\\test\\jupyter_landmarks.csv",
                           usecols = range(1, 209))

y_test_data = pd.read_csv(".\\samples\\test\\unity_blendshapes.csv",
                           usecols = range(2, 74))


x_test_tensor = torch.tensor(x_test_data.values, requires_grad=True).float()

for i in range(x_test_tensor.shape[1]):
    sdev_norm = torch.std(x_test_tensor[:, i])
    sdev_norm = 0.001 if sdev_norm == 0 else sdev_norm
    x_test_tensor[:, i] = (x_test_tensor[:, i] - torch.mean(x_test_tensor[:, i])) / sdev_norm

y_test_tensor = torch.tensor(y_test_data.values, requires_grad=True).float()

sdev_history = []
mean_history = []


for i in range(y_test_tensor.shape[1]):
    sdev_norm = torch.std(y_test_tensor[:, i])
    sdev_norm = 0.001 if sdev_norm == 0 else sdev_norm
    
#     sdev_history.append(sdev_norm)
#     mean_history.append(torch.mean(y_train_tensor[:, i]))
    
    y_test_tensor[:, i] = (y_test_tensor[:, i] - torch.mean(y_test_tensor[:, i])) / sdev_norm


class LandmarksDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y


    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    
    def __len__(self):
        return len(self.x)

train_dataset = LandmarksDataset(x_train_tensor, y_train_tensor)
test_dataset = LandmarksDataset(x_test_tensor, y_test_tensor)

In [37]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd


def create_nn(batch_size=50, learning_rate=0.001, epochs=15,
              log_interval=10):
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True)

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False)

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(208, 1024)
            self.fc2 = nn.Linear(1024, 512)
            self.fc3 = nn.Linear(512, 256)
            self.fc4 = nn.Linear(256, 128)
            self.fc5 = nn.Linear(128, 72)
            
        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = F.relu(self.fc4(x))
            x = self.fc5(x)
            return x

    net = Net()

    # create a stochastic gradient descent optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    # create a loss function
    # criterion = nn.L1Loss(reduction='mean')
    criterion = nn.MSELoss(reduction='mean')

    # run the main training loop
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()))

    # run a test loop
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        net_out = net(data)

        # sum up batch loss
        test_loss += criterion(net_out, target).item()
#         pred = net_out.data.max(1)[1]  # get the index of the max log-probability
#         correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}'.format(
        test_loss))

if __name__ == "__main__":
    run_opt = 2
    if run_opt == 1:
        simple_gradient()
    elif run_opt == 2:
        create_nn()

Train Epoch: 0 [0/3103 (0%)]	Loss: 1.018746
Train Epoch: 0 [500/3103 (16%)]	Loss: 0.881655
Train Epoch: 0 [1000/3103 (32%)]	Loss: 0.963214
Train Epoch: 0 [1500/3103 (48%)]	Loss: 0.876432
Train Epoch: 0 [2000/3103 (63%)]	Loss: 0.947020
Train Epoch: 0 [2500/3103 (79%)]	Loss: 0.885221
Train Epoch: 0 [3000/3103 (95%)]	Loss: 0.926224
Train Epoch: 1 [0/3103 (0%)]	Loss: 0.924159
Train Epoch: 1 [500/3103 (16%)]	Loss: 0.869222
Train Epoch: 1 [1000/3103 (32%)]	Loss: 0.908651
Train Epoch: 1 [1500/3103 (48%)]	Loss: 0.806775
Train Epoch: 1 [2000/3103 (63%)]	Loss: 0.742221
Train Epoch: 1 [2500/3103 (79%)]	Loss: 0.882386
Train Epoch: 1 [3000/3103 (95%)]	Loss: 0.754821
Train Epoch: 2 [0/3103 (0%)]	Loss: 0.802896
Train Epoch: 2 [500/3103 (16%)]	Loss: 0.740963
Train Epoch: 2 [1000/3103 (32%)]	Loss: 0.770917
Train Epoch: 2 [1500/3103 (48%)]	Loss: 0.631123
Train Epoch: 2 [2000/3103 (63%)]	Loss: 0.761894
Train Epoch: 2 [2500/3103 (79%)]	Loss: 0.734278
Train Epoch: 2 [3000/3103 (95%)]	Loss: 0.643194
Train E